# H2O Model

 * Wrap a H2O model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * [H2O](https://www.h2o.ai/download/)

```bash
pip install seldon-core
pip install sklearn
pip install grpcio-tools
```

## Train locally
 

In [ ]:
!mkdir -p experiment

In [ ]:
import h2o
h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
path = "http://s3.amazonaws.com/h2o-public-test-data/smalldata/prostate/prostate.csv.zip"
h2o_df = h2o.import_file(path)
h2o_df['CAPSULE'] = h2o_df['CAPSULE'].asfactor()
model = H2OGeneralizedLinearEstimator(family = "binomial")
model.train(y = "CAPSULE",
            x = ["AGE", "RACE", "PSA", "GLEASON"],
            training_frame = h2o_df)
modelfile = model.download_mojo(path="./experiment/", get_genmodel_jar=False)
print("Model saved to " + modelfile)


In [ ]:
!mv experiment/*.zip src/main/resources/model.zip

Wrap model using s2i

In [ ]:
!s2i build . seldonio/seldon-core-s2i-java-build:0.1 h2o-test:0.1 --runtime-image seldonio/seldon-core-s2i-java-runtime:0.1

In [ ]:
!docker run --name "h2o_predictor" -d --rm -p 5000:5000 h2o-test:0.1

Send some random features that conform to the contract

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm h2o_predictor --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-java-build:0.1 h2o-test:0.1 --runtime-image seldonio/seldon-core-s2i-java-runtime:0.1

In [ ]:
!kubectl create -f h2o_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete